1. conda create --prefix "C:\\Users\\rs659\\Desktop\\Object-Detection\\wincondaprojenv" python=3.11.6  
2. conda activate "C:\\Users\\rs659\\Desktop\\Object-Detection\\wincondaprojenv"
3. pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
4. conda install -p c:\Users\rs659\Desktop\Object-Detection\wincondaprojenv ipykernel --update-deps --force-reinstall


pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
conda install -p c:\Users\rs659\Desktop\Object-Detection\wincondaprojenv ipykernel --update-deps --force-reinstall

5. pip install pybind11
6. pip install opencv-python
7. pip instal ultralytics
8. pip install pylabel




Merge Different COCO dataset
https://gradiant.github.io/pyodi/reference/apps/coco-merge/
pyodi coco merge coco_1.json coco_2.json output.json


In [ ]:
import os
import zipfile
from pylabel import importer
import shutil

In [ ]:
import os
from pylabel import importer

def import_dataset(path_to_annotations, path_to_images, dataset_name):
    dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name=dataset_name)
    return dataset

def split_and_export(dataset, output_path):
    # dataset.splitter.GroupShuffleSplit(train_pct=.6, val_pct=.2, test_pct=.2)
    dataset.export.ExportToYoloV5(output_path=output_path, yaml_file='dataset.yaml', copy_images=True)

# For train dataset
train_annotations = "COCOAnnotations/train.json"
train_images = os.getcwd() + "/MergeDataset/"
train_dataset = import_dataset(train_annotations, train_images, "BCCD_coco")
split_and_export(train_dataset, 'YOLO_dataset_train/labels')

# For test dataset
test_annotations = "COCOAnnotations/test.json"
test_images = os.getcwd() + "/MergeDataset/"  # Assuming test images are in the same folder as train images
test_dataset = import_dataset(test_annotations, test_images, "BCCD_coco")
split_and_export(test_dataset, 'YOLO_dataset_test/labels')


In [ ]:
import os
import shutil
from pathlib import Path

# Define source and destination paths
test_source = 'YOLO_dataset_test/labels'
train_source = 'YOLO_dataset_train/labels'
test_dest = 'YOLO_dataset/labels/test'
train_dest = 'YOLO_dataset/labels/train'
source_file = "COCOAnnotations/dataset.yaml"
destination_dir = "YOLO_dataset"

# Create main directories
os.makedirs('YOLO_dataset/images/train', exist_ok=True)
os.makedirs('YOLO_dataset/images/test', exist_ok=True)

# Copy images from test and train directories
shutil.copytree('YOLO_dataset_test/images', 'YOLO_dataset/images/test', dirs_exist_ok=True)
shutil.copytree('YOLO_dataset_train/images', 'YOLO_dataset/images/train', dirs_exist_ok=True)

# Copy label directories
try:
    shutil.copytree(test_source, test_dest)
    shutil.copytree(train_source, train_dest)
    print("Directories copied successfully!")
except FileExistsError:
    print("Destination directories already exist. Remove or rename them and try again.")
except Exception as e:
    print(f"An error occurred: {e}")

# Copy dataset.yaml file
shutil.copy(source_file, destination_dir)

shutil.rmtree('YOLO_dataset_test')
shutil.rmtree('YOLO_dataset_train')

In [4]:
import os
from ultralytics import YOLO

ROOT_DIR = '/Users/rishabhshah/Desktop/AIPI590/Object-Detection/YOLO_dataset'
# ROOT_DIR = os.getcwd() + 'YOLO_dataset'


# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO('yolov8n.pt')


# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "dataset.yaml"), epochs=1,project='saved_models/YOLO/')  # train the model 